In [1]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2023 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################

In [2]:
# Import Pyomo libraries
from pyomo.environ import Constraint, exp, Param, Set, units as pyunits, Var

# Import IDAES cores
from idaes.core import (
    declare_process_block_class,
    MaterialFlowBasis,
    ReactionParameterBlock,
    ReactionBlockDataBase,
    ReactionBlockBase,
)
from idaes.core.util.constants import Constants as const
import idaes.logger as idaeslog

In [3]:
units_metadata = {
    "time": pyunits.s,
    "length": pyunits.m,
    "mass": pyunits.kg,
    "amount": pyunits.mol,
    "temperature": pyunits.K,
}

properties_metadata = {
    "k_rxn": {"method": None},
    "k_eq": {"method": None},
    "reaction_rate": {"method": None},
}

In [4]:
def define_kinetic_reactions(self):
    # Rate Reaction Index
    self.rate_reaction_idx = Set(initialize=["R1"])

    # Rate Reaction Stoichiometry
    self.rate_reaction_stoichiometry = {
        ("R1", "Vap", "benzene"): 1,
        ("R1", "Vap", "toluene"): -1,
        ("R1", "Vap", "hydrogen"): -1,
        ("R1", "Vap", "methane"): 1,
        ("R1", "Vap", "diphenyl"): 0,
    }

In [5]:
def define_equilibrium_reactions(self):
    # Equilibrium Reaction Index
    self.equilibrium_reaction_idx = Set(initialize=["E1"])

    # Equilibrium Reaction Stoichiometry
    self.equilibrium_reaction_stoichiometry = {
        ("E1", "Vap", "benzene"): -2,
        ("E1", "Vap", "toluene"): 0,
        ("E1", "Vap", "hydrogen"): 1,
        ("E1", "Vap", "methane"): 0,
        ("E1", "Vap", "diphenyl"): 1,
    }

In [6]:
def define_parameters(self):
    # Arrhenius Constant
    self.arrhenius = Param(
        default=1.25e-9,
        doc="Arrhenius constant",
        units=pyunits.mol / pyunits.m**3 / pyunits.s / pyunits.Pa**2,
    )

    # Activation Energy
    self.energy_activation = Param(
        default=3800, doc="Activation energy", units=pyunits.J / pyunits.mol
    )

In [7]:
@declare_process_block_class("HDAReactionParameterBlock")
class HDAReactionParameterData(ReactionParameterBlock):
    """
    Reaction Parameter Block Class
    """

    def build(self):
        """
        Callable method for Block construction.
        """
        super(HDAReactionParameterData, self).build()

        self._reaction_block_class = HDAReactionBlock

        define_kinetic_reactions(self)
        define_equilibrium_reactions(self)
        define_parameters(self)

    @classmethod
    def define_metadata(cls, obj):
        obj.add_properties(properties_metadata)
        obj.add_default_units(units_metadata)

In [8]:
def define_variables_and_parameters(self):
    self.k_rxn = Var(
        initialize=7e-10,
        doc="Rate constant",
        units=pyunits.mol / pyunits.m**3 / pyunits.s / pyunits.Pa**2,
    )

    self.reaction_rate = Var(
        self.params.rate_reaction_idx,
        initialize=0,
        doc="Rate of reaction",
        units=pyunits.mol / pyunits.m**3 / pyunits.s,
    )

    self.k_eq = Param(initialize=10000, doc="Equlibrium constant", units=pyunits.Pa)

In [9]:
def define_rate_expression(self):
    self.arrhenius_equation = Constraint(
        expr=self.k_rxn
        == self.params.arrhenius
        * exp(
            -self.params.energy_activation
            / (const.gas_constant * self.state_ref.temperature)
        )
    )

    def rate_rule(b, r):
        return b.reaction_rate[r] == (
            b.k_rxn
            * b.state_ref.mole_frac_comp["toluene"]
            * b.state_ref.mole_frac_comp["hydrogen"]
            * b.state_ref.pressure**2
        )

    self.rate_expression = Constraint(self.params.rate_reaction_idx, rule=rate_rule)

In [10]:
def define_equilibrium_expression(self):
    self.equilibrium_constraint = Constraint(
        expr=self.k_eq
        * self.state_ref.mole_frac_comp["benzene"]
        * self.state_ref.pressure
        == self.state_ref.mole_frac_comp["diphenyl"]
        * self.state_ref.mole_frac_comp["hydrogen"]
        * self.state_ref.pressure**2
    )

In [11]:
class _HDAReactionBlock(ReactionBlockBase):
    def initialize(blk, outlvl=idaeslog.NOTSET, **kwargs):
        init_log = idaeslog.getInitLogger(blk.name, outlvl, tag="properties")
        init_log.info("Initialization Complete.")


@declare_process_block_class("HDAReactionBlock", block_class=_HDAReactionBlock)
class HDAReactionBlockData(ReactionBlockDataBase):
    def build(self):

        super(HDAReactionBlockData, self).build()

        define_variables_and_parameters(self)
        define_rate_expression(self)
        define_equilibrium_expression(self)

    def get_reaction_rate_basis(b):
        return MaterialFlowBasis.molar

In [12]:
from pyomo.environ import ConcreteModel
from pyomo.util.check_units import assert_units_consistent

from idaes.core import FlowsheetBlock
from idaes.core.solvers import get_solver
from idaes.models.unit_models import CSTR

from thermophysical_property_example import HDAParameterBlock

from idaes.core.util.model_statistics import degrees_of_freedom

In [13]:
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False)

m.fs.thermo_params = HDAParameterBlock()
m.fs.reaction_params = HDAReactionParameterBlock(property_package=m.fs.thermo_params)

m.fs.reactor = CSTR(
    property_package=m.fs.thermo_params,
    reaction_package=m.fs.reaction_params,
    has_equilibrium_reactions=True,
)

In [14]:
print("Degrees of Freedom: ", degrees_of_freedom(m))

Degrees of Freedom:  9


In [15]:
m.fs.reactor.inlet.flow_mol.fix(100)
m.fs.reactor.inlet.temperature.fix(500)
m.fs.reactor.inlet.pressure.fix(350000)
m.fs.reactor.inlet.mole_frac_comp[0, "benzene"].fix(0.1)
m.fs.reactor.inlet.mole_frac_comp[0, "toluene"].fix(0.4)
m.fs.reactor.inlet.mole_frac_comp[0, "hydrogen"].fix(0.4)
m.fs.reactor.inlet.mole_frac_comp[0, "methane"].fix(0.1)
m.fs.reactor.inlet.mole_frac_comp[0, "diphenyl"].fix(0.0)

m.fs.reactor.volume.fix(1)

print("Degrees of Freedom: ", degrees_of_freedom(m))

Degrees of Freedom:  0


In [16]:
m.fs.reactor.initialize(
    state_args={
        "flow_mol": 100,
        "mole_frac_comp": {
            "benzene": 0.15,
            "toluene": 0.35,
            "hydrogen": 0.35,
            "methane": 0.15,
            "diphenyl": 0.01,
        },
        "temperature": 600,
        "pressure": 350000,
    }
)

solver = get_solver()
results = solver.solve(m, tee=True)

2023-02-18 12:05:03 [INFO] idaes.init.fs.reactor.control_volume.properties_in: Properties Initialized optimal - Optimal Solution Found.


2023-02-18 12:05:03 [INFO] idaes.init.fs.reactor.control_volume.properties_out: Properties Initialized optimal - Optimal Solution Found.


2023-02-18 12:05:03 [INFO] idaes.init.fs.reactor.control_volume.reactions: Initialization Complete.


2023-02-18 12:05:03 [INFO] idaes.init.fs.reactor.control_volume: Initialization Complete


2023-02-18 12:05:03 [INFO] idaes.init.fs.reactor: Initialization Complete: optimal - Optimal Solution Found


Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale


Number of Iterations....: 0

                                   (scaled)                 (unscaled)
Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
Dual infeasibility......:   0.0000000000000000e+00    0.0000000000000000e+00
Constraint violation....:   7.9734232250544974e-12    1.3038516044616698e-08
Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
Overall NLP error.......:   7.9734232250544974e-12    1.3038516044616698e-08


Number of objective function evaluations             = 1
Number of objective gradient evaluations             = 1
Number of equality constraint evaluations            = 1
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 1
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 0
Total CPU secs in IPOPT (w/o function evaluations)   =      0.001
Total CPU secs in NLP function evaluations     

In [17]:
m.fs.reactor.report()


Unit : fs.reactor                                                          Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key    : Value  : Units      : Fixed : Bounds
    Volume : 1.0000 : meter ** 3 :  True : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                Units         Inlet     Outlet  
    flow_mol                 mole / second     100.00     100.00
    mole_frac_comp benzene   dimensionless    0.10000    0.15963
    mole_frac_comp toluene   dimensionless    0.40000    0.31243
    mole_frac_comp methane   dimensionless    0.10000    0.18757
    mole_frac_comp hydrogen  dimensionless    0.40000    0.32640
    mole_frac_comp diphenyl  dimensionless     0.0000   0.013973
    temperature                     kelvin     500.00     790.21
    pressure                        pascal 3.5000e+05 3.5000e

In [18]:
assert_units_consistent(m)